In [10]:
from flask import Flask, jsonify, render_template
import pandas as pd
import json

app = Flask(__name__)

# 全局數據框
data_draw_CD_name = pd.DataFrame({
    'Group': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Scan_M': [1, 2, 1, 2, 1, 2],
    'L1': [10, 12, 11, 13, 14, 15],
    'L2': [7, 9, 8, 10, 10, 12]
})

@app.route('/')
def index():
    return render_template('details.html', draw_columns=['L1', 'L2'])

@app.route('/draw_line/<string:column>', methods=['GET'])
def draw_line(column):
    global data_draw_CD_name

    if column not in data_draw_CD_name.columns:
        return "Column not found!", 404

    # 根據 Group 列分組
    grouped = data_draw_CD_name.groupby('Group')

    # 用於生成圖表數據的列表
    series_data = [{
        'name': group_name,
        'data': group_data[column].tolist()
    } for group_name, group_data in grouped]

    x_data = data_draw_CD_name['Scan_M'].tolist()

    chart_html = f'''
    <figure class="highcharts-figure" style="text-align: left;">
        <div id="myChart" style="height: 600px; width: 900px"></div>
    </figure>
    <script type="text/javascript">
        Highcharts.chart("myChart", {{
            chart: {{
                type: "line",
                zoomType: "xy"
            }},
            title: {{
                text: "Line Plot of {column}"
            }},
            xAxis: {{
                categories: {json.dumps(x_data)},
                title: {{
                    text: "Scan_M"
                }}
            }},
            yAxis: {{
                title: {{
                    text: "{column} Value"
                }}
            }},
            series: {json.dumps(series_data)}
        }});
    </script>
    '''
    
    return chart_html


if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Oct/2024 17:28:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2024 17:28:38] "GET /draw_line/L1 HTTP/1.1" 200 -


In [9]:
from flask import Flask, render_template
import pandas as pd
import json

app = Flask(__name__)

# 全局數據框
data_draw_CD_name = None

@app.route('/')
def index():
    global data_draw_CD_name
    data_draw_CD_name = pd.DataFrame({
        'Group': ['A', 'A', 'B', 'B', 'C', 'C'],
        'Scan_M': [1, 2, 1, 2, 1, 2],
        'L1': [10, 12, 11, 13, 14, 15],
        'L2': [7, 9, 8, 10, 10, 12]
    })
    
    return render_template('details.html', draw_columns=['L1', 'L2'])

@app.route('/draw_line/<string:column>', methods=['GET'])
def draw_line(column):
    global data_draw_CD_name

    if data_draw_CD_name is None or data_draw_CD_name.empty:
        return "Data not available!", 404

    if column not in data_draw_CD_name.columns:
        return "Column not found!", 404

    # 根據 Group 列分組
    grouped = data_draw_CD_name.groupby('Group')

    # 用於生成圖表數據的列表
    series_data = []
    for group_name, group_data in grouped:
        series_data.append({
            'name': group_name,  # 使用 Group 名稱作為系列名稱
            'value': group_data[column].tolist()  # 使用該分組的 column 數據
        })

    x_data = data_draw_CD_name['Scan_M'].tolist()  # 獲取 X 軸數據

    chart_html = f'''
    <div id="myChart" style="height: 600px; width: 900px;"></div>
    <script type="text/javascript">
        var myChart = echarts.init(document.getElementById('myChart'));
        var option = {{
            title: {{
                text: 'Line Plot of {column}'
            }},
            tooltip: {{ trigger: 'axis' }},
            legend: {{
                data: {json.dumps([group['name'] for group in series_data])}
            }},
            xAxis: {{
                type: 'category',
                data: {json.dumps(x_data)}
            }},
            yAxis: {{
                type: 'value'
            }},
            series: [
                {','.join([f'{{name: "{group["name"]}", type: "line", data: {json.dumps(group["value"])}}}' for group in series_data])}
            ]
        }};
        myChart.setOption(option);
    </script>
    '''

    return chart_html

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Oct/2024 16:48:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2024 16:48:17] "GET /draw_line/L1 HTTP/1.1" 200 -


In [44]:
from flask import Flask, render_template, request
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

app = Flask(__name__)

# 全局數據框
data_draw_CD_name = None

@app.route('/')
def index():
    global data_draw_CD_name
    
    data_draw_CD_name = pd.DataFrame({
    'Scan_M': ['Scan3_M3->0', 'Scan2_M4->0', 'Scan4_M2->0', 'Scan6_M2->1', 'Scan6_M3->0', 'Scan3_M3->0', 'Scan1_M4->0', 'Scan3_M1->0', 'Scan6_M3->1', 'Scan6_M4->0'],
    'L1': [21.67, 21.41, 21.87, 21.76, 21.76, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L2': [31.39, 31.25, 31.69, 31.53, 31.51, 21.67, 21.41, 21.87, 21.76, 21.76],
    'L3': [13.63, 13.37, 13.79, 13.69, 13.73, 13.63, 13.37, 13.79, 13.69, 13.73],
    'L4': [13.67, 13.38, 13.86, 13.76, 13.77, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L5': [-0.21, 0.11, -0.22, -0.02, 0.01, 0.30, -0.11, 0.23, 0.10, 0.09],
    'A1': [0.30, -0.11, 0.23, 0.10, 0.09, -0.21, 0.11, -0.22, -0.02, 0.01],
    'Group': ['Left', 'Left-Center', 'Center', 'Right-Center', 'Right', 'Left', 'Left-Center', 'Center', 'Right-Center', 'Right']  # Group 列
})
    print(data_draw_CD_name)

    # 預設顯示第一個列
    default_column = 'L1'
    chart_html = generate_plot(default_column)

    return render_template('details.html', draw_columns=['L1', 'L2'], chart_html=chart_html, selected_column=default_column)

@app.route('/select_column', methods=['POST'])
def select_column():
    global data_draw_CD_name
    selected_column = request.form.get('column')

    # 生成選中的列的圖表
    chart_html = generate_plot(selected_column)
    
    return render_template('details.html', draw_columns=['L1', 'L2'], chart_html=chart_html, selected_column=selected_column)

def generate_plot(column):
    global data_draw_CD_name

    if data_draw_CD_name is None or data_draw_CD_name.empty:
        return "Data not available!"

    if column not in data_draw_CD_name.columns:
        return "Column not found!"

    # 根據 Group 列分組
    grouped = data_draw_CD_name.groupby('Group')

    # 創建每個 Group 的數據線
    lines = []
    for group_name, group_data in grouped:
        lines.append(go.Scatter(
            x=group_data['Scan_M'],
            y=group_data[column],
            mode='lines+markers',
            name=f'{group_name}'
        ))

    # 設定圖表布局
    layout = go.Layout(
        title=f'Line Plot of {column}',
        xaxis=dict(title='Scan_M'),
        yaxis=dict(title=column),
        hovermode='closest'
    )

    # 組合數據與布局
    fig = go.Figure(data=lines, layout=layout)

    # 生成圖表的 HTML
    return pio.to_html(fig, full_html=False)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [12/Oct/2024 23:37:26] "GET / HTTP/1.1" 200 -


        Scan_M     L1     L2     L3     L4    L5    A1         Group
0  Scan3_M3->0  21.67  31.39  13.63  13.67 -0.21  0.30          Left
1  Scan2_M4->0  21.41  31.25  13.37  13.38  0.11 -0.11   Left-Center
2  Scan4_M2->0  21.87  31.69  13.79  13.86 -0.22  0.23        Center
3  Scan6_M2->1  21.76  31.53  13.69  13.76 -0.02  0.10  Right-Center
4  Scan6_M3->0  21.76  31.51  13.73  13.77  0.01  0.09         Right
5  Scan3_M3->0  13.67  21.67  13.63  13.67  0.30 -0.21          Left
6  Scan1_M4->0  13.38  21.41  13.37  13.38 -0.11  0.11   Left-Center
7  Scan3_M1->0  13.86  21.87  13.79  13.86  0.23 -0.22        Center
8  Scan6_M3->1  13.76  21.76  13.69  13.76  0.10 -0.02  Right-Center
9  Scan6_M4->0  13.77  21.76  13.73  13.77  0.09  0.01         Right


127.0.0.1 - - [12/Oct/2024 23:37:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Oct/2024 23:37:28] "POST /select_column HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2024 23:37:31] "POST /select_column HTTP/1.1" 200 -


In [71]:
from flask import Flask, render_template, request
import pandas as pd
import plotly.graph_objs as go
import plotly.io as pio

app = Flask(__name__)

data_draw_CD_name = None #設定全域變數

@app.route('/')
def index():
    global data_draw_CD_name
    
    data_draw_CD_name = pd.DataFrame({
    'Point_No': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Scan_M': ['Scan3_M3->0', 'Scan2_M4->0', 'Scan4_M2->0', 'Scan6_M2->1', 'Scan6_M3->0', 'Scan3_M3->1', 'Scan1_M4->0', 'Scan3_M1->0', 'Scan6_M3->1', 'Scan6_M4->0'],
    'L1': [21.67, 21.41, 21.87, 21.76, 21.76, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L2': [31.39, 31.25, 31.69, 31.53, 31.51, 21.67, 21.41, 21.87, 21.76, 21.76],
    'L3': [13.58, 13.37, 13.79, 13.69, 13.73, 13.63, 13.37, 13.79, 13.69, 13.73],
    'L4': [13.67, 13.38, 13.86, 13.76, 13.77, 13.67, 13.38, 13.86, 13.76, 13.77],
    'A1': [-0.21, 0.11, -0.22, -0.02, 0.01, 0.30, -0.11, 0.23, 0.10, 0.09],
    'A2': [0.30, -0.11, 0.23, 0.10, 0.09, -0.21, 0.11, -0.22, -0.02, 0.01],
    'Group': ['Left', 'Left-Center', 'Center', 'Right-Center', 'Right', 'Left', 'Left-Center', 'Center', 'Right-Center', 'Right']
})
    # 預設顯示第一個列和所有Group
    default_column = 'L1'
    all_groups = data_draw_CD_name['Group'].unique().tolist()
    chart_html = generate_plot(default_column, all_groups)

    return render_template('details.html', draw_columns=['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2'], groups=all_groups, chart_html=chart_html, selected_column=default_column, selected_groups=all_groups)

@app.route('/select_column', methods=['POST'])
def select_column():
    global data_draw_CD_name
    selected_column = request.form.get('column')

    # 獲取選擇的 Group
    selected_groups = request.form.getlist('groups')

    # 生成選中的列的圖表
    chart_html = generate_plot(selected_column, selected_groups)
    
    all_groups = data_draw_CD_name['Group'].unique().tolist()
    return render_template('details.html', draw_columns=['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2'], groups=all_groups, chart_html=chart_html, selected_column=selected_column, selected_groups=selected_groups)

def generate_plot(column, selected_groups):
    global data_draw_CD_name

    if data_draw_CD_name is None or data_draw_CD_name.empty:
        return "Data not available!"

    if column not in data_draw_CD_name.columns:
        return "Column not found!"
    
    # 提取出 Scan 和 M 中的數字部分，將其作為排序依據
    data_draw_CD_name['Sort_Scan'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('Scan')[1].split('_')[0]))
    data_draw_CD_name['Sort_M'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('M')[1].split('->')[0]))

    # 按照 Sort_Scan 和 Sort_M 列進行排序
    data_draw_CD_name = data_draw_CD_name.sort_values(by=['Sort_Scan', 'Sort_M'])
    print(data_draw_CD_name)

    # 根據 Group 列分組並篩選出選擇的 groups
    grouped = data_draw_CD_name.groupby('Group')
    
    # 創建每個選中的 Group 的數據線
    lines = []
    for group_name, group_data in grouped:
        if group_name in selected_groups:
            lines.append(go.Scatter(
                x=group_data['Scan_M'],
                y=group_data[column],
                mode='lines+markers',
                name=f'{group_name}',
                hovertemplate=(
                    'Point No: %{customdata[0]}<br>' +
                    'Scan_M: %{x}<br>' +
                     column + ': %{y}<extra></extra>'
                ),
                customdata=group_data[['Point_No']].values  # 傳遞 Point_No 作為自定義數據
            ))

    # 設定圖表布局，強制 Scan_M 顯示順序
    layout = go.Layout(
        title=f'Line Plot of {column}',
        xaxis=dict(
            title='Scan_M',
            categoryorder='array',  # 使用自定義排序順序
            categoryarray=data_draw_CD_name['Scan_M'],  # 依照排序後的 Scan_M 顯示
        ),
        yaxis=dict(title=column),
        hovermode='closest'
    )

    # 組合數據與布局
    fig = go.Figure(data=lines, layout=layout)

    # 生成圖表的 HTML
    return pio.to_html(fig, full_html=False)

if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


   Point_No       Scan_M     L1     L2     L3     L4    A1    A2  \
6         7  Scan1_M4->0  13.38  21.41  13.37  13.38 -0.11  0.11   
1         2  Scan2_M4->0  21.41  31.25  13.37  13.38  0.11 -0.11   
7         8  Scan3_M1->0  13.86  21.87  13.79  13.86  0.23 -0.22   
0         1  Scan3_M3->0  21.67  31.39  13.58  13.67 -0.21  0.30   
5         6  Scan3_M3->1  13.67  21.67  13.63  13.67  0.30 -0.21   
2         3  Scan4_M2->0  21.87  31.69  13.79  13.86 -0.22  0.23   
3         4  Scan6_M2->1  21.76  31.53  13.69  13.76 -0.02  0.10   
4         5  Scan6_M3->0  21.76  31.51  13.73  13.77  0.01  0.09   
8         9  Scan6_M3->1  13.76  21.76  13.69  13.76  0.10 -0.02   
9        10  Scan6_M4->0  13.77  21.76  13.73  13.77  0.09  0.01   

          Group  Sort_Scan  Sort_M  
6   Left-Center          1       4  
1   Left-Center          2       4  
7        Center          3       1  
0          Left          3       3  
5          Left          3       3  
2        Center          4   

127.0.0.1 - - [13/Oct/2024 21:26:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 21:26:39] "GET /favicon.ico HTTP/1.1" 404 -


In [62]:
scan_m = 'Scan4_M2->3'

int(scan_m.split('M')[1].split('->')[0])

2

In [88]:
import pandas as pd

# 假設你的 data_draw_CD_name 已經有以下的數據
data_draw_CD_name = pd.DataFrame({
    'Point_No': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Scan_M': ['Scan3_M3->0', 'Scan2_M4->0', 'Scan4_M2->0', 'Scan6_M2->1', 'Scan6_M3->0', 'Scan3_M3->1', 'Scan1_M4->0', 'Scan3_M1->0', 'Scan6_M3->1', 'Scan6_M4->0'],
    'L1': [21.67, 21.41, 21.87, 21.76, 21.76, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L2': [31.39, 31.25, 31.69, 31.53, 31.51, 21.67, 21.41, 21.87, 21.76, 21.76],
    'L3': [13.58, 13.37, 13.79, 13.69, 13.73, 13.63, 13.37, 13.79, 13.69, 13.73],
    'L4': [13.67, 13.38, 13.86, 13.76, 13.77, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L5': [-0.21, 0.11, -0.22, -0.02, 0.01, 0.30, -0.11, 0.23, 0.10, 0.09],
    'A1': [0.30, -0.11, 0.23, 0.10, 0.09, -0.21, 0.11, -0.22, -0.02, 0.01],
    'A2': [0.10, -0.12, 0.24, 0.11, 0.08, -0.22, 0.12, -0.23, -0.01, 0.02]
})

# 提取出 Scan 和 M 中的數字部分，將其作為排序依據
data_draw_CD_name['Sort_Scan'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('Scan')[1].split('_')[0]))
data_draw_CD_name['Sort_M'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('M')[1].split('->')[0]))

# 按照 Sort_Scan 和 Sort_M 列進行排序
data_draw_CD_name = data_draw_CD_name.sort_values(by=['Sort_Scan', 'Sort_M']).reset_index(drop=True)
print(data_draw_CD_name)

# 創建一個空的 DataFrame 來存儲結果
data_draw_DCD = pd.DataFrame(columns=['Diff_Scan_M'] + [f'Diff-{col}' for col in ['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']])

# 遍歷 DataFrame，計算當 Scan_M 變化時的差異
for i in range(1, len(data_draw_CD_name)):
    current_scan = data_draw_CD_name.iloc[i]['Scan_M']
    previous_scan = data_draw_CD_name.iloc[i-1]['Scan_M']
    
    # 提取 Scan 和 M 的編號
    current_scan_number = current_scan.split('_')[0]
    previous_scan_number = previous_scan.split('_')[0]
    current_M = current_scan.split('_')[1].split('->')[0]
    previous_M = previous_scan.split('_')[1].split('->')[0]

    # 判斷是否需要顯示兩個 Scan 還是只顯示一個 Scan（M 不同但 Scan 相同）
    if current_scan_number != previous_scan_number:
        diff_scan_M = f"{previous_scan.split('->')[0]}->{current_scan.split('->')[0]}"
    else:
        diff_scan_M = f"{previous_scan.split('_')[0]}_{previous_M}->{current_M}"
    
    # 計算 L1 到 A2 的差異
    diff_values = data_draw_CD_name.iloc[i][['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']].values - data_draw_CD_name.iloc[i-1][['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']].values
    
    # 創建一行數據，包括 Diff_Scan_M 和差異數據
    diff_row = pd.Series([diff_scan_M] + diff_values.tolist(), index=data_draw_DCD.columns)
    
    # 將結果附加到 data_draw_DCD
    data_draw_DCD = data_draw_DCD.append(diff_row, ignore_index=True)

# 顯示結果
print(data_draw_DCD)


   Point_No       Scan_M     L1     L2     L3     L4    L5    A1    A2  \
0         7  Scan1_M4->0  13.38  21.41  13.37  13.38 -0.11  0.11  0.12   
1         2  Scan2_M4->0  21.41  31.25  13.37  13.38  0.11 -0.11 -0.12   
2         8  Scan3_M1->0  13.86  21.87  13.79  13.86  0.23 -0.22 -0.23   
3         1  Scan3_M3->0  21.67  31.39  13.58  13.67 -0.21  0.30  0.10   
4         6  Scan3_M3->1  13.67  21.67  13.63  13.67  0.30 -0.21 -0.22   
5         3  Scan4_M2->0  21.87  31.69  13.79  13.86 -0.22  0.23  0.24   
6         4  Scan6_M2->1  21.76  31.53  13.69  13.76 -0.02  0.10  0.11   
7         5  Scan6_M3->0  21.76  31.51  13.73  13.77  0.01  0.09  0.08   
8         9  Scan6_M3->1  13.76  21.76  13.69  13.76  0.10 -0.02 -0.01   
9        10  Scan6_M4->0  13.77  21.76  13.73  13.77  0.09  0.01  0.02   

   Sort_Scan  Sort_M  
0          1       4  
1          2       4  
2          3       1  
3          3       3  
4          3       3  
5          4       2  
6          6       2  
7

In [94]:
import pandas as pd

# 假設你的 data_draw_CD_name 已經有以下的數據
data_draw_CD_name = pd.DataFrame({
    'Point_No': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'Scan_M': ['Scan3_M3->0', 'Scan2_M4->0', 'Scan4_M2->0', 'Scan6_M2->1', 'Scan6_M3->0', 'Scan3_M3->1', 'Scan1_M4->0', 'Scan3_M1->0', 'Scan6_M3->1', 'Scan6_M4->0'],
    'L1': [21.67, 21.41, 21.87, 21.76, 21.76, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L2': [31.39, 31.25, 31.69, 31.53, 31.51, 21.67, 21.41, 21.87, 21.76, 21.76],
    'L3': [13.58, 13.37, 13.79, 13.69, 13.73, 13.63, 13.37, 13.79, 13.69, 13.73],
    'L4': [13.67, 13.38, 13.86, 13.76, 13.77, 13.67, 13.38, 13.86, 13.76, 13.77],
    'L5': [-0.21, 0.11, -0.22, -0.02, 0.01, 0.30, -0.11, 0.23, 0.10, 0.09],
    'A1': [0.30, -0.11, 0.23, 0.10, 0.09, -0.21, 0.11, -0.22, -0.02, 0.01],
    'A2': [0.10, -0.12, 0.24, 0.11, 0.08, -0.22, 0.12, -0.23, -0.01, 0.02]
})

# 提取出 Scan 和 M 中的數字部分，將其作為排序依據
data_draw_CD_name['Sort_Scan'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('Scan')[1].split('_')[0]))
data_draw_CD_name['Sort_M'] = data_draw_CD_name['Scan_M'].apply(lambda x: int(x.split('M')[1].split('->')[0]))

# 按照 Sort_Scan 和 Sort_M 列進行排序
data_draw_CD_name = data_draw_CD_name.sort_values(by=['Sort_Scan', 'Sort_M']).reset_index(drop=True)
print(data_draw_CD_name)

# 創建一個空的 DataFrame 來存儲結果
data_draw_DCD = pd.DataFrame(columns=['Diff_Scan_M'] + [f'Diff-{col}' for col in ['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']])

# 遍歷 DataFrame，計算當 Scan_M 變化時的差異
for i in range(1, len(data_draw_CD_name)):
    current_scan = data_draw_CD_name.iloc[i]['Scan_M']
    previous_scan = data_draw_CD_name.iloc[i-1]['Scan_M']
    
    # 提取 Scan 和 M 的編號
    current_scan_number = current_scan.split('_')[0]
    previous_scan_number = previous_scan.split('_')[0]
    current_M = current_scan.split('_')[1].split('->')[0]
    previous_M = previous_scan.split('_')[1].split('->')[0]
    
    # 跳過 Scan 和 M 相同的情況，不進行相減
    if current_scan_number == previous_scan_number and current_M == previous_M:
        continue
    
    # 判斷是否需要顯示兩個 Scan 還是只顯示一個 Scan（M 不同但 Scan 相同）
    if current_scan_number != previous_scan_number:
        diff_scan_M = f"{previous_scan.split('->')[0]}->{current_scan.split('->')[0]}"
    else:
        diff_scan_M = f"{previous_scan.split('_')[0]}_{previous_M}->{current_M}"
    
    # 計算 L1 到 A2 的差異
    diff_values = data_draw_CD_name.iloc[i][['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']].values - data_draw_CD_name.iloc[i-1][['L1', 'L2', 'L3', 'L4', 'L5', 'A1', 'A2']].values
    
    # 創建一行數據，包括 Diff_Scan_M 和差異數據
    diff_row = pd.Series([diff_scan_M] + diff_values.tolist(), index=data_draw_DCD.columns)
    
    # 將結果附加到 data_draw_DCD
    data_draw_DCD = data_draw_DCD.append(diff_row, ignore_index=True)

# 顯示結果
print(data_draw_DCD)


   Point_No       Scan_M     L1     L2     L3     L4    L5    A1    A2  \
0         7  Scan1_M4->0  13.38  21.41  13.37  13.38 -0.11  0.11  0.12   
1         2  Scan2_M4->0  21.41  31.25  13.37  13.38  0.11 -0.11 -0.12   
2         8  Scan3_M1->0  13.86  21.87  13.79  13.86  0.23 -0.22 -0.23   
3         1  Scan3_M3->0  21.67  31.39  13.58  13.67 -0.21  0.30  0.10   
4         6  Scan3_M3->1  13.67  21.67  13.63  13.67  0.30 -0.21 -0.22   
5         3  Scan4_M2->0  21.87  31.69  13.79  13.86 -0.22  0.23  0.24   
6         4  Scan6_M2->1  21.76  31.53  13.69  13.76 -0.02  0.10  0.11   
7         5  Scan6_M3->0  21.76  31.51  13.73  13.77  0.01  0.09  0.08   
8         9  Scan6_M3->1  13.76  21.76  13.69  13.76  0.10 -0.02 -0.01   
9        10  Scan6_M4->0  13.77  21.76  13.73  13.77  0.09  0.01  0.02   

   Sort_Scan  Sort_M  
0          1       4  
1          2       4  
2          3       1  
3          3       3  
4          3       3  
5          4       2  
6          6       2  
7